## CSC 215 - Project 2
### Khoi Hoang
### William Dobson

In [3]:
import os
import csv
import json
import numpy as np
import pandas as pd
from scipy.stats import zscore
from sklearn import preprocessing
from sklearn.model_selection import train_test_split
from sklearn import metrics
from matplotlib.pyplot import figure, show
import matplotlib.pyplot as plt
%matplotlib inline
import collections
import sklearn.feature_extraction.text as sk_text

In [4]:
# Encode text values to dummy variables(i.e. [1,0,0],[0,1,0],[0,0,1] for red,green,blue)
def encode_text_dummy(df, name):
    dummies = pd.get_dummies(df[name])
    for x in dummies.columns:
        dummy_name = "{}-{}".format(name, x)
        df[dummy_name] = dummies[x]
    df.drop(name, axis=1, inplace=True)

# Encode text values to indexes(i.e. [1],[2],[3] for red,green,blue).
def encode_text_index(df, name):
    le = preprocessing.LabelEncoder()
    df[name] = le.fit_transform(df[name])
    return le.classes_

# Encode a numeric column as zscores
def encode_numeric_zscore(df, name, mean=None, sd=None):
    if mean is None:
        mean = df[name].mean()

    if sd is None:
        sd = df[name].std()

    df[name] = (df[name] - mean) / sd
    
# Convert all missing values in the specified column to the median
def missing_median(df, name):
    med = df[name].median()
    df[name] = df[name].fillna(med)    

# Regression chart.
def chart_regression(pred,y,sort=True):
    t = pd.DataFrame({'pred' : pred, 'y' : y.flatten()})
    if sort:
        t.sort_values(by=['y'],inplace=True)
    a = plt.plot(t['y'].tolist(),label='expected')
    b = plt.plot(t['pred'].tolist(),label='prediction')
    plt.ylabel('output')
    plt.legend()
    plt.show()    
    
# Convert a Pandas dataframe to the x,y inputs that TensorFlow needs
def to_xy(df, target):
    result = []
    for x in df.columns:
        if x != target:
            result.append(x)
    # find out the type of the target column. 
    target_type = df[target].dtypes
    target_type = target_type[0] if isinstance(target_type, collections.Sequence) else target_type
    # Encode to int for classification, float otherwise. TensorFlow likes 32 bits.
    if target_type in (np.int64, np.int32):
        # Classification
        dummies = pd.get_dummies(df[target])
        return df[result].values.astype(np.float32), dummies.values.astype(np.float32)
    else:
        # Regression
        return df[result].values.astype(np.float32), df[target].values.astype(np.float32)


### Convert JSON data into tabular format for Pandas

In [ ]:
#reviews.json
outfile = open("review_stars.tsv", 'w')
sfile = csv.writer(outfile, delimiter ="\t", quoting=csv.QUOTE_MINIMAL)
sfile.writerow(['business_id','stars', 'text'])
with open('review.json', encoding="utf8") as f:
    for line in f:
        row = json.loads(line)
        # some special char must be encoded in 'utf-8'
        sfile.writerow([row['business_id'], row['stars'], (row['text']).encode('utf-8')])

outfile.close()

In [ ]:
#business.json
outfile = open("business.tsv", 'w')
sfile = csv.writer(outfile, delimiter ="\t", quoting=csv.QUOTE_MINIMAL)
sfile.writerow(['business_id','stars', 'review_count', 'categories'])
with open('business.json', encoding="utf8") as f:
    for line in f:
        row = json.loads(line)
        # some special char must be encoded in 'utf-8'
        sfile.writerow([row['business_id'], row['stars'], row['review_count'], row['categories']])

outfile.close()

In [ ]:
#tip.json
outfile = open("tip.tsv", 'w')
sfile = csv.writer(outfile, delimiter ="\t", quoting=csv.QUOTE_MINIMAL)
sfile.writerow(['business_id','text'])
with open('tip.json', encoding="utf8") as f:
    for line in f:
        row = json.loads(line)
        # some special char must be encoded in 'utf-8'
        sfile.writerow([row['business_id'], (row['text']).encode('utf-8')])

outfile.close()

In [5]:
df_review= pd.read_csv('review_stars.tsv', delimiter ="\t", encoding="utf-8")
df_business= pd.read_csv('business.tsv', delimiter ="\t", encoding="utf-8")
df_tip= pd.read_csv('tip.tsv', delimiter ="\t", encoding="utf-8")
df_tip[0:5]

,business_id,text
0,VaKXUpmWTTWDKbpJ3aQdMw,"b'Great for watching games, ufc, and whatever ..."
1,OPiPeoJiv92rENwbq76orA,b'Happy Hour 2-4 daily with 1/2 price drinks a...
2,5KheTjYPu1HcQzQFtm4_vw,b'Good chips and salsa. Loud at times. Good se...
3,TkoyGi8J7YFjA6SbaRzrxg,b'The setting and decoration here is amazing. ...
4,AkL6Ous6A1atZejfZXn1Bg,b'Molly is definately taking a picture with Sa...


In [6]:
df_review.head()

,business_id,stars,text
0,ujmEBvifdJM6h6RLv4wQIg,1.0,b'Total bill for this horrible service? Over $...
1,NZnhc2sEQy3RmzKTZnqtwQ,5.0,"b""I *adore* Travis at the Hard Rock's new Kell..."
2,WTqjgwHlXbSFevF32_DJVw,5.0,"b""I have to say that this office really has it..."
3,ikCg8xy5JIg_NGPx-MSIDA,5.0,"b""Went in for a lunch. Steak sandwich was deli..."
4,b1b1eb3uo-w561D0ZfCEiQ,1.0,b'Today was my second out of three sessions I ...


In [7]:
#only consider reviews that have more than 50 characters
df_review = df_review[df_review['text'].str.len() >= 50]
df_review.shape

(6678404, 3)

In [8]:
df_tip = df_tip[df_tip['text'].str.len() >= 20]
df_tip.shape

(1094522, 2)

In [9]:
#group all reviews by each business and create a new dataframe, where each line is a business with all its reviews
df_review_agg = df_review.groupby('business_id')['text'].sum()

df_all_reviews = pd.DataFrame({'business_id': df_review_agg.index, 'all_reviews': df_review_agg.values})

In [10]:
#group all tip by each business and create a new dataframe, where each line is a business with all its tips
df_tip_agg = df_tip.groupby('business_id')['text'].sum()

df_all_tips = pd.DataFrame({'business_id': df_tip_agg.index, 'all_tips': df_tip_agg.values})

In [11]:
df_all_tips.head()

,business_id,all_tips
0,--1UhMGODdWsrMastO9DZw,"b""Delicious! One of the best burritos, salsa a..."
1,--6MefnULPED_I942VcFNA,b'BBQ pork is sold out early on Saturday'b'The...
2,--7zmmkVg-IMGaXbuVd0SQ,"b""Good place to stop when traffic don't go.""b'..."
3,--9QQLMTbFzLJ_oT-ON3Xw,"b""Tip? Don't get your hair cut here. If they'r..."
4,--9e1ONYQuAa-CB_Rrw7Tw,b'Wine pairings for my prix fixe meal. Mmmm'b...


#### Only consider businesses which have more than 20 reviews

In [12]:
df_business = df_business[df_business['review_count'] >= 20]
df_business.shape

(57644, 4)

In [13]:
#merge df_business, df_all_reviews, and df_all_tips by business_id
df_ready_for_sklearn = pd.merge(pd.merge(df_all_reviews, df_business, on='business_id', how='inner'), df_all_tips, on='business_id', how='inner')
df_ready_for_sklearn.shape

(54657, 6)

In [14]:
#label encode the stars
le = preprocessing.LabelEncoder()
df_ready_for_sklearn['encoded_stars'] = le.fit_transform(df_ready_for_sklearn['stars'])

In [15]:
print(df_ready_for_sklearn.shape)
df_ready_for_sklearn[0:5]

(54657, 7)


,business_id,all_reviews,stars,review_count,categories,all_tips,encoded_stars
0,--1UhMGODdWsrMastO9DZw,b'If you are looking for authentic Mexican str...,4.0,24,"Restaurants, Mexican","b""Delicious! One of the best burritos, salsa a...",6
1,--6MefnULPED_I942VcFNA,"b""They have the best Chinese BBQ Pork (Char Si...",3.0,44,"Chinese, Restaurants",b'BBQ pork is sold out early on Saturday'b'The...,4
2,--7zmmkVg-IMGaXbuVd0SQ,"b""I'm a tad reluctant to write a review as I r...",4.0,58,"Breweries, Food","b""Good place to stop when traffic don't go.""b'...",6
3,--9e1ONYQuAa-CB_Rrw7Tw,b'Very busy and noisy restaurant.\nAsparagas w...,4.0,1613,"Cajun/Creole, Seafood, Steakhouses, Restaurants",b'Wine pairings for my prix fixe meal. Mmmm'b...,6
4,--DaPTJW3-tB1vP-PfdTEg,b'My sister and in laws were in town and we wa...,3.5,49,"Restaurants, Breakfast & Brunch",b'Awesome. Huge portions definitely worth a v...,5


## Split the data and prepare for training and testing

In [17]:
x_train, x_test, y_train, y_test = train_test_split(df_ready_for_sklearn, df_ready_for_sklearn['encoded_stars'], test_size=0.25, random_state=43)

In [22]:
x_train['review_count_zscore'] = zscore(x_train['review_count'])
x_train[0:5]

C:\Users\hgdan\Anaconda3\envs\tf-gpu\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


,business_id,all_reviews,stars,review_count,categories,all_tips,encoded_stars,review_count_zscore
53526,ygcX18rboNOaYnNmRCDytw,b'I drive 45 minutes from Cleveland to enjoy t...,4.0,30,"Restaurants, Cafes, Tea Rooms, Bubble Tea, Food",b'cute shop and helpful staff!'b'The greatest ...,6,-0.374402
16481,IB_aESRBh2cmYWSF6Fv0jQ,b'I was really happy with Spencer\'s when I pu...,3.0,94,"Home & Garden, Shopping, Appliances, Electronics",b'Best pricing in town.Next day delivery great...,4,-0.021752
21744,OI_lS_aamMgjnHqNlaO2rA,"b""For a post office, this location is pretty g...",2.5,21,"Public Services & Government, Local Services, ...","b""Kinda old school. Feels weird. Have to Stand...",3,-0.423993
49426,tyQWsZUSoMuoX6_5pcCE8Q,"b""Honestly the worst latte I've ever had. Got ...",4.5,108,"Coffee & Tea, Coffee Roasteries, Food",b'Cute simple coffee shop. Great coffee and th...,7,0.055390
23280,Q7rrhu3FJxTASTo46pT9vA,"b""It tastes great and the atmosphere is great ...",3.5,101,"Coffee & Tea, Food",b'$3 cappuccino is a pretty sweet deal. I have...,5,0.016819


In [23]:
x_test['review_count_zscore'] = zscore(x_test['review_count'])
x_test[0:5]

C:\Users\hgdan\Anaconda3\envs\tf-gpu\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


,business_id,all_reviews,stars,review_count,categories,all_tips,encoded_stars,review_count_zscore
53763,yyiHkabQBtLifVTsO1r1kw,"b""I am not the biggest fan of Walmart but they...",3.0,37,"Food, Shopping, Grocery, Fashion, Drugstores, ...",b'Picking up some goodies for E.B.!'b'Rich wal...,4,-0.293785
50802,vZu12JAudoXBv3-i0JmSiQ,"b""I brought a watch strap in to get a few link...",3.0,29,"Shopping, Jewelry, Watch Repair, Local Service...",b'5 mins in and out for a battery replacement....,4,-0.330296
54033,zIpW3YlE9fFwyWOT5HGI6A,"b""Only giving one star cuz I have to. Terrible...",2.0,41,"Fast Food, Ice Cream & Frozen Yogurt, Restaura...","b""Get better employees and listen to what peop...",2,-0.275530
27076,UaL6yRGSv9fYCyn2DJLu8w,"b'Greek salad add avacado, very fresh very del...",4.5,42,"Food, Juice Bars & Smoothies, Restaurants, Cof...","b'Love this place. Good food, fresh as fuck ju...",7,-0.270966
14056,FML0YqZ7dLBBC3Suap4_4Q,"b""My favourite place in the city! Great vibe a...",4.0,296,"Nightlife, Restaurants, Tapas/Small Plates, Bars",b'Most of the food is made in house'b'All is g...,6,0.888237


In [24]:
#obtain tfidf score for reviews
vectorizer = sk_text.TfidfVectorizer(stop_words='english', max_features=250, min_df=10)

reviews_matrix_train = vectorizer.fit_transform(x_train['all_reviews'])
reviews_tfidf_train = reviews_matrix_train.toarray()

reviews_matrix_test = vectorizer.transform(x_test['all_reviews'])
reviews_tfidf_test = reviews_matrix_test.toarray()

vectorizer.get_feature_names()

['10',
 '15',
 '20',
 '30',
 'able',
 'absolutely',
 'actually',
 'amazing',
 'area',
 'ask',
 'asked',
 'atmosphere',
 'away',
 'awesome',
 'bad',
 'bar',
 'beef',
 'beer',
 'best',
 'better',
 'big',
 'bit',
 'bread',
 'breakfast',
 'burger',
 'business',
 'busy',
 'called',
 'came',
 'car',
 'care',
 'check',
 'cheese',
 'chicken',
 'clean',
 'coffee',
 'come',
 'coming',
 'cooked',
 'cool',
 'couldn',
 'couple',
 'cream',
 'customer',
 'day',
 'decent',
 'decided',
 'definitely',
 'delicious',
 'dessert',
 'did',
 'didn',
 'different',
 'dinner',
 'disappointed',
 'dish',
 'dishes',
 'don',
 'drink',
 'drinks',
 'eat',
 'eating',
 'end',
 'enjoy',
 'enjoyed',
 'especially',
 'excellent',
 'experience',
 'extra',
 'family',
 'fantastic',
 'far',
 'fast',
 'favorite',
 'feel',
 'felt',
 'finally',
 'fish',
 'flavor',
 'food',
 'free',
 'fresh',
 'fried',
 'friend',
 'friendly',
 'friends',
 'fries',
 'fun',
 'gave',
 'getting',
 'going',
 'good',
 'got',
 'great',
 'half',
 'happy',


In [25]:
#obtain tfidf score for tips
vectorizer = sk_text.TfidfVectorizer(stop_words='english', max_features=250, min_df=10)

tips_matrix_train = vectorizer.fit_transform(x_train['all_tips'])
tips_tfidf_train = tips_matrix_train.toarray()

tips_matrix_test = vectorizer.transform(x_test['all_tips'])
tips_tfidf_test = tips_matrix_test.toarray()

vectorizer.get_feature_names()

['10',
 '15',
 '20',
 '30',
 '50',
 '99',
 'absolutely',
 'amazing',
 'area',
 'ask',
 'atmosphere',
 'authentic',
 'away',
 'awesome',
 'bad',
 'bar',
 'bbq',
 'beautiful',
 'beef',
 'beer',
 'best',
 'better',
 'big',
 'bit',
 'bomb',
 'bread',
 'breakfast',
 'bring',
 'brunch',
 'buffet',
 'burger',
 'burgers',
 'business',
 'busy',
 'buy',
 'came',
 'car',
 'card',
 'cash',
 'cheap',
 'check',
 'cheese',
 'chicken',
 'chips',
 'chocolate',
 'clean',
 'close',
 'closed',
 'coffee',
 'cold',
 'come',
 'coming',
 'cool',
 'cream',
 'customer',
 'day',
 'deal',
 'decent',
 'definitely',
 'delicious',
 'did',
 'didn',
 'dinner',
 'does',
 'don',
 'drink',
 'drinks',
 'drive',
 'early',
 'eat',
 'enjoy',
 'excellent',
 'experience',
 'extra',
 'family',
 'fantastic',
 'far',
 'fast',
 'favorite',
 'feel',
 'fish',
 'flavor',
 'food',
 'free',
 'fresh',
 'friday',
 'fried',
 'friendly',
 'friends',
 'fries',
 'fun',
 'getting',
 'going',
 'good',
 'got',
 'great',
 'green',
 'half',
 'hap

In [26]:
temp = x_train['review_count_zscore'].values
input_data_train = np.column_stack((reviews_tfidf_train, tips_tfidf_train))
input_data_train = np.column_stack((input_data_train, temp))
input_data_train.shape

temp = x_test['review_count_zscore'].values
input_data_test = np.column_stack((reviews_tfidf_test, tips_tfidf_test))
input_data_test = np.column_stack((input_data_test, temp))
input_data_test.shape

(13665, 501)

In [34]:
#convert into dataframe
df_train = pd.DataFrame(input_data_train)
df_test = pd.DataFrame(input_data_test)
df_train['encoded_stars'] = y_train
df_test['encoded_stars'] = y_test
missing_median(df_train, 'encoded_stars')
missing_median(df_test, 'encoded_stars')

In [48]:
print(df_train.shape)
df_train.head()

(40992, 502)


,0,1,2,3,4,5,6,7,8,9,...,492,493,494,495,496,497,498,499,500,encoded_stars
0,0.015470,0.000000,0.000000,0.000000,0.000000,0.008722,0.008023,0.038333,0.000000,0.016279,...,0.000000,0.000000,0.000000,0.0,0.0,0.000000,0.0,0.0,-0.374402,5.0
1,0.026053,0.039063,0.025437,0.051188,0.042569,0.020985,0.019302,0.014756,0.007397,0.050916,...,0.000000,0.180343,0.000000,0.0,0.0,0.174842,0.0,0.0,-0.021752,5.0
2,0.000000,0.050186,0.024510,0.049322,0.024610,0.024264,0.044636,0.000000,0.021381,0.022643,...,0.000000,0.000000,0.000000,0.0,0.0,0.000000,0.0,0.0,-0.423993,6.0
3,0.007468,0.005806,0.008506,0.002853,0.011388,0.019649,0.012909,0.044409,0.034628,0.005239,...,0.000000,0.000000,0.073175,0.0,0.0,0.000000,0.0,0.0,0.055390,6.0
4,0.005317,0.000000,0.006057,0.000000,0.021285,0.005996,0.024818,0.018446,0.015851,0.041966,...,0.053099,0.000000,0.000000,0.0,0.0,0.000000,0.0,0.0,0.016819,5.0


In [42]:
x_train_tf,y_train_tf = to_xy(df_train,'encoded_stars')

In [51]:
x_test_tf,y_test_tf = to_xy(df_test,'encoded_stars')

# Tensorflow

In [50]:
from keras.models import Sequential
from keras.layers.core import Dense, Activation
from keras.callbacks import EarlyStopping
from keras.callbacks import ModelCheckpoint
from keras import optimizers

In [53]:
model = Sequential()

model.add(Dense(50, input_dim=x_train_tf.shape[1], activation='relu')) # Hidden 1     #  why input_dim=x.shape[1]?  
model.add(Dense(200, activation='relu')) # Hidden 2
model.add(Dense(100, activation='relu')) # Hidden 3
model.add(Dense(10, activation='relu')) # Hidden 4
model.add(Dense(1)) # Output

model.compile(loss='mean_squared_error', optimizer='adam')

monitor = EarlyStopping(monitor='val_loss', min_delta=1e-3, patience=5, verbose=1, mode='auto')
checkpointer = ModelCheckpoint(filepath=".\lowest_error.hdf5", verbose=0, save_best_only=True) # save best model


model.fit(x_train_tf,y_train_tf, validation_data=(x_test_tf,y_test_tf), callbacks=[monitor,checkpointer],verbose=2,epochs=1000)    # Verbosity mode. 0 = silent, 1 = progress bar, 2 = one line per epoch.


Train on 40992 samples, validate on 13665 samples
Epoch 1/1000
 - 5s - loss: 2.2889 - val_loss: 0.6606
Epoch 2/1000
 - 5s - loss: 1.8741 - val_loss: 0.7530
Epoch 3/1000
 - 5s - loss: 1.8288 - val_loss: 0.6542
Epoch 4/1000
 - 5s - loss: 1.8086 - val_loss: 0.6526
Epoch 5/1000
 - 5s - loss: 1.7790 - val_loss: 0.8803
Epoch 6/1000
 - 5s - loss: 1.7336 - val_loss: 0.7170
Epoch 7/1000
 - 5s - loss: 1.6856 - val_loss: 0.8002
Epoch 8/1000
 - 5s - loss: 1.6189 - val_loss: 0.8295
Epoch 9/1000
 - 5s - loss: 1.5575 - val_loss: 0.9083
Epoch 00009: early stopping


In [ ]:
pred = model.predict(x_test)
print("Shape: {}".format(pred.shape))
print(pred)

In [ ]:
score = np.sqrt(metrics.mean_squared_error(pred,y_test))
print("Final score (RMSE): {}".format(score))

In [ ]:
# Plot the chart
chart_regression(pred.flatten(),y_test)

### Use tfidfVectorizer to obtain TFIDF representation for each business

In [ ]:
#obtain tfidf score for reviews
vectorizer = sk_text.TfidfVectorizer(max_features=250, min_df=10)

reviews_matrix = vectorizer.fit_transform(df_ready_for_sklearn['all_reviews'])

reviews_tfidf = reviews_matrix.toarray()

In [ ]:
print(reviews_tfidf.shape)
vectorizer.get_feature_names()

In [ ]:
#obtain tfidf score for tips
vectorizer = sk_text.TfidfVectorizer(max_features=250, min_df=10)

tips_matrix = vectorizer.fit_transform(df_ready_for_sklearn['all_tips'])

tips_tfidf = tips_matrix.toarray()

In [ ]:
print(tips_tfidf[0:5])
vectorizer.get_feature_names()

# Linear Regression

Preparing data for training/testing

In [ ]:
#Merge reviews_tfidf, tips_tfidf and review_count into one array for input
temp = df_ready_for_sklearn['review_count_zscore'].values
input_data = np.column_stack((reviews_tfidf, tips_tfidf))
input_data = np.column_stack((input_data, temp))
input_data.shape

In [ ]:
#split into train/test data
x_train, x_test, y_train, y_test = train_test_split(input_data, df_ready_for_sklearn['stars'], test_size=0.25, random_state=43)

In [ ]:
test_data = input_data[test_loc[0]]
for i in range(1, len(test_loc)):
    test_data = np.append(test_data, input_data[test_loc[i]], axis=0)
test_data.shape

In [ ]:
print(x_train.shape)
print(x_test.shape)
print(y_train.shape)
print(y_test.shape)

In [ ]:
from sklearn.linear_model import LinearRegression

# instantiate the model (using the default parameters)
linreg = LinearRegression()

# fit the model with data
linreg.fit(x_train, y_train)

# predict the response for new observations
y_pred_Linear = linreg.predict(test_data)

In [ ]:
print("Prediction result: ")
print("Business ID                         Categories             Predicted Star Rating         True Star Rating")
for i in range(5):
    print("{:.20}             {:.23}         {:.5}                         {}".format(test_businesses[i]['business_id'], test_businesses[i]['categories'], y_pred_Linear[i], test_true_stars[i]))

### Support Vector Machine

In [ ]:
#split into train/test data
#using encoded stars because classification problem
x_train, x_test, y_train, y_test = train_test_split(input_data, df_ready_for_sklearn['encoded_stars'], test_size=0.25, random_state=43)

In [ ]:
print(x_train.shape)
print(x_test.shape)
print(y_train.shape)
print(y_test.shape)

In [ ]:
from sklearn.svm import LinearSVC

clf = LinearSVC(dual=False)
clf.fit(x_train, y_train)

y_pred_SVM = clf.predict(test_data)

In [ ]:
y_pred_SVM = le.inverse_transform(y_pred_SVM)
print("Prediction result: ")
print("Business ID                         Categories             Predicted Star Rating         True Star Rating")
for i in range(5):
    print("{:.20}             {:.23}         {:.5}                         {}".format(test_businesses[i]['business_id'], test_businesses[i]['categories'], y_pred_SVM[i], test_true_stars[i]))

### Logistic Regression

In [ ]:
from sklearn.linear_model import LogisticRegression

# instantiate the model (using the default parameters)
logreg = LogisticRegression()

# fit the model with data
logreg.fit(x_train, y_train)

# predict the response for new observations
y_pred_Logistic = logreg.predict(test_data)

In [ ]:
y_pred_Logistic = le.inverse_transform(y_pred_Logistic)
print("Prediction result: ")
print("Business ID                         Categories             Predicted Star Rating         True Star Rating")
for i in range(5):
    print("{:.20}             {:.23}         {:.5}                         {}".format(test_businesses[i]['business_id'], test_businesses[i]['categories'], y_pred_Logistic[i], test_true_stars[i]))

### Nearest Neighbor

In [ ]:
from sklearn.neighbors import KNeighborsClassifier

# instantiate the model (using the value K=10)
knn = KNeighborsClassifier(n_neighbors=10)

# fit the model with data
knn.fit(x_train, y_train)

In [ ]:
# predict the response for new observations
y_neighbor = knn.predict(test_data)

In [ ]:
y_neighbor = le.inverse_transform(y_neighbor)
print("Prediction result: ")
print("Business ID                         Categories             Predicted Star Rating         True Star Rating")
for i in range(5):
    print("{:.20}             {:.23}         {:.5}                         {}".format(test_businesses[i]['business_id'], test_businesses[i]['categories'], y_neighbor[i], test_true_stars[i]))

### Multinomial Naive Bayes

In [ ]:
from sklearn.preprocessing import MinMaxScaler

scaler = MinMaxScaler()

temp = scaler.fit_transform(df_ready_for_sklearn['review_count'].values.reshape(1, -1))

input_data_MNB = np.column_stack((reviews_tfidf, tips_tfidf))
input_data_MNB = np.column_stack((input_data_MNB, temp.reshape(-1,1)))

input_data_MNB

In [ ]:
test_data = input_data_MNB[test_loc[0]]
for i in range(1, len(test_loc)):
    test_data = np.append(test_data, input_data_MNB[test_loc[i]], axis=0)
test_data.shape

In [ ]:
x_train, x_test, y_train, y_test = train_test_split(input_data_MNB, df_ready_for_sklearn['encoded_stars'], test_size=0.25, random_state=43)

In [ ]:
from sklearn.naive_bayes import MultinomialNB

# instantiate the model (using the default parameters)
mnb = MultinomialNB()

# fit the model with data
mnb.fit(x_train, y_train)

# predict the response for new observations
y_mnb = mnb.predict(test_data)

In [ ]:
y_mnb = le.inverse_transform(y_mnb)
print("Prediction result: ")
print("Business ID                         Categories             Predicted Star Rating         True Star Rating")
for i in range(5):
    print("{:.20}             {:.23}         {:.5}                         {}".format(test_businesses[i]['business_id'], test_businesses[i]['categories'], y_mnb[i], test_true_stars[i]))

# TensorFlow

In [49]:
from keras.models import Sequential
from keras.layers.core import Dense, Activation
from keras.callbacks import EarlyStopping
from keras.callbacks import ModelCheckpoint
from keras import optimizers

Using TensorFlow backend.


In [ ]:
#obtain tfidf score for reviews
vectorizer = sk_text.TfidfVectorizer(stop_words='english', max_features=250, min_df=10)

reviews_matrix_train = vectorizer.fit_transform(x_train['all_reviews'])
reviews_tfidf_train = reviews_matrix_train.toarray()

reviews_matrix_test = vectorizer.transform(x_test['all_reviews'])
reviews_tfidf_test = reviews_matrix_test.toarray()

vectorizer.get_feature_names()

In [ ]:
#obtain tfidf score for tips
vectorizer = sk_text.TfidfVectorizer(stop_words='english', max_features=250, min_df=10)

tips_matrix_train = vectorizer.fit_transform(x_train['all_tips'])
tips_tfidf_train = tips_matrix_train.toarray()

tips_matrix_test = vectorizer.transform(x_test['all_tips'])
tips_tfidf_test = tips_matrix_test.toarray()

vectorizer.get_feature_names()

In [ ]:
temp = x_train['review_count_zscore'].values
input_data_train = np.column_stack((reviews_tfidf_train, tips_tfidf_train))
input_data_train = np.column_stack((input_data_train, temp))
input_data_train.shape

temp = x_test['review_count_zscore'].values
input_data_test = np.column_stack((reviews_tfidf_test, tips_tfidf_test))
input_data_test = np.column_stack((input_data_test, temp))
input_data_test.shape

In [ ]:
df_train = pd.DataFrame(input_data_train)
df_test = pd.DataFrame(input_data_test)
df_train['encoded_stars'] = y_train
df_test['encoded_stars'] = y_test
missing_median(df_train, 'encoded_stars')
missing_median(df_test, 'encoded_stars')

In [ ]:
df_train.shape

In [ ]:
y_train.shape

In [ ]:
y_train.isnull().values.any()

In [ ]:
x_train,y_train = to_xy(df_train,'encoded_stars')

In [ ]:
# Encode to a 2D matrix for training
x,y = to_xy(df_train,'stars')

In [ ]:
x.shape

In [ ]:
y.shape

In [ ]:
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.25, random_state=43)

In [ ]:
model = Sequential()

model.add(Dense(501, input_dim=x_train.shape[1], activation='relu')) # Hidden 1     #  why input_dim=x.shape[1]?  
model.add(Dense(200, activation='relu')) # Hidden 2
model.add(Dense(100, activation='relu')) # Hidden 3
model.add(Dense(10, activation='relu')) # Hidden 4
model.add(Dense(1)) # Output

model.compile(loss='mean_squared_error', optimizer='adam')

monitor = EarlyStopping(monitor='loss', min_delta=1e-3, patience=3, verbose=1, mode='auto')
checkpointer = ModelCheckpoint(filepath=".\lowest_error.hdf5", verbose=0, save_best_only=True) # save best model


model.fit(x_train,y_train,callbacks=[monitor,checkpointer],verbose=2,epochs=100)    # Verbosity mode. 0 = silent, 1 = progress bar, 2 = one line per epoch.


In [ ]:
pred = model.predict(x_test)
print("Shape: {}".format(pred.shape))
print(pred)

In [ ]:
score = np.sqrt(metrics.mean_squared_error(pred,y_test))
print("Final score (RMSE): {}".format(score))

In [ ]:
# Plot the chart
chart_regression(pred.flatten(),y_test)